# Open Source Chatbots 101: Start Talking in Minutes
**By Alan Meigs alan.meigs@gmail.com**

- [Github Repos](https://github.com/Drakosfire?tab=repositories)

- [DungeonMind - Generative AI Tools for Dungeons and Dragons](https://www.DungeonMind.net)

- [BlueSky](@dungeonmind.bsky.social)

- [LinkedIn](https://www.linkedin.com/in/alan-meigs/)

This notebook was created for the [Software Freedom School](https://sofree.us/) Longmont in person Meetup on 2/1/2025.

Shortened URL to this Notebook https://shorturl.at/2E4Pt


## Agenda


1.   Introduction
2.   How to use a notebook
    - a. Getting Started
    - b. Prepare the Enviornment
3.   Basic Q&A Bot
4.   Retrieval Augmented Generation
5.   Function Calling
6.   Vocabulary
7.   License



# Introduction

Hello! My name is Alan Meigs.

I have been many things, from teacher, to marijuana farmer, to now aspiring developer. I fell in love with generative AI through pure spite and skepticism. My friends showed me AI generated images of DnD characters in the spring of 2023 and I thought they looked awful. In an effort to prove Generative AI was not the next big thing I learned how to run models on my powerful gaming computer. I quickly went from unimpressed to genuinely inspired, moving from generating images to training models.

Since then I have been coding daily, expanding my skillset from novice with python to proficient, and teaching myself JavaScript and TypeScript. I went from someone who dreamed of reading research papers, to comfortably following along with some of the densest cutting edge papers. I've completed multiple projects combining image and text generation that you can find on my website DungeonMind. All of my code is open source, so you can completely duplicate almost anything I've built.  

## Getting Started with This Notebook

Welcome to this hands-on tutorial, where we’ll walk through the essentials of spinning up an open-source Large Language Model (LLM) environment in Google Colab and quickly start chatting with it. Our focus is on leveraging entirely open-source solutions, from downloading the model weights to running inference in your own Colab session. By the end, you’ll:

- Understand how to prepare a Colab environment for LLM deployment.
- See how to load and run a quantized model (in GGUF format) locally.
- Explore chat features like streaming responses and memory.
- Get a quick introduction to Retrieval-Augmented Generation (RAG) workflows.
- Take a look at function calling concepts with open-source LLMs.

1. **Review the Notebook**: You’ll see text explanations (like this) and blocks of code.
2. **Run a Code Cell**: Hover your mouse over the left side of a code cell. Click the “▶” (play) button that appears, or press **Shift+Enter** on your keyboard. This will execute the code in that cell.
3. **Proceed in Order**: Move to the next cell after the current one finishes running. If any cell asks for input, follow the instructions right in the cell’s output area.
 - a) **Caveat** This notebook assumes you are working in Google Colab and downloading the model for long term storage to google Drive. If you do not want to use Google Drive you will have to download the model everytime you use the notebook.  
4. **Read the Text**: Above or below each cell, you’ll see explanations and directions on what the code does. This helps you understand the process step by step.
5. **Explore**: Feel free to edit or re-run cells multiple times. Experimenting is the best way to learn!

That’s it! Click “▶” (play) on the first cell to begin.


##**Prepare the Environment**
Enable Free Tier GPU in Colab

- Click “Runtime” in the Colab menu bar.
Select “Change runtime type.”
Choose “GPU” under Hardware accelerator (T4 GPU is typical in the free tier).

Why?

- RAM Limits: Google Colab’s free tier offers roughly 12–16GB of system RAM. This will limit the size of LLMs you can run. Models in the range of 7B to 13B parameters are typically feasible. Larger models (30B, 70B, etc.) generally won’t fit in the free tier.
GPU Memory: A T4 GPU often has 12–15GB of VRAM. This will affect how many layers you can offload to the GPU (or if you can offload them all).

Below, we confirm that we do indeed have a GPU (like a T4) available:

###**Check to ensure CUDA drivers are installed**

In [ ]:
!nvidia-smi


Sat Feb  1 16:45:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Assign File Paths
import os, sys
nb_path = '/content/notebooks'

###**Mount Google Drive for Storage**

**Optional This is to shorten the download time for the model ~6 Gb if you intend to use this notebook multiple times. **

By mounting Google Drive, you can store model files and avoid redownloading them each session. This is especially handy with quantized model files that can still be several gigabytes.

- Mount the drive, and create a symlink (shortcut) to simplify referencing the Colab Notebooks folder.

In [ ]:
# Optional if you want to save the model
from google.colab import drive
drive.mount('/content/drive')
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###**Install llama.cpp for model inference**

- llama-cpp-python is a Python wrapper for the llama.cpp project.
We choose a specific wheel (in this example, CUDA 12.4 support) that matches the T4 runtime on Colab.
This will also install huggingface-hub and other dependencies.

GGUF Format

The GGUF format is a quantized model format used by llama.cpp. It drastically reduces model size and memory usage, enabling us to run 7B–13B models on consumer hardware or limited Colab GPU memory.

After installing, we can verify the version with:

In [ ]:
#Install llama-cpp-python this is a tested and verified build that works with Colab
!pip install --no-cache-dir llama-cpp-python==0.2.77 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
import llama_cpp
print("llama_cpp version:", llama_cpp.__version__)

!pip show llama-cpp-python

llama_cpp version: 0.2.77
Name: llama_cpp_python
Version: 0.2.77
Summary: Python bindings for the llama.cpp library
Home-page: https://github.com/abetlen/llama-cpp-python
Author: 
Author-email: Andrei Betlen <abetlen@gmail.com>
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: diskcache, jinja2, numpy, typing-extensions
Required-by: 


###**Download the Model**

Here, we download the OpenCoder-8B-Instruct-Q6_K_L.gguf model from Hugging Face if it’s not already present in our notebooks folder:

- Check if the file (.gguf) already exists.
If not, we download from Hugging Face using the ```hf_hub_download``` function.
This step can take several minutes, depending on file size (typically multiple gigabytes).




In [ ]:
from huggingface_hub import hf_hub_download
import os

file_name = "OpenCoder-8B-Instruct-Q6_K_L.gguf"
llm_file_path = os.path.join(nb_path, file_name)

if os.path.isfile(llm_file_path):
    print(f"'{file_name}' is already present in '{nb_path}'.")
else:
    print(f"'{file_name}' not found. Downloading now...")
    llm_model_path = hf_hub_download(
        repo_id="bartowski/OpenCoder-8B-Instruct-GGUF",
        filename=file_name,
        revision="main",
        local_dir=nb_path
    )
    print("Model downloaded to:", llm_model_path)


'OpenCoder-8B-Instruct-Q6_K_L.gguf' is already present in '/content/notebooks'.


###**Mount the Model**

This step loads the quantized model weights into memory. Once done, we can query it.

In [ ]:
from llama_cpp import Llama

llm = Llama(
      model_path=llm_file_path,
      n_gpu_layers=-1, # -1 means all layers offloaded to GPU
      # n_ctx=2048, # Uncomment to increase the context window
)


llama_model_loader: loaded meta data with 41 key-value pairs and 291 tensors from /content/notebooks/OpenCoder-8B-Instruct-Q6_K_L.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = OpenCoder 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = OpenCoder
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str              = other
llama_model_loader: - kv   7:     

## **Basic Q&A Bot**

Testing the model with a simple prompt:

- We ask, “What are you?”
- We limit ```max_tokens``` to 32.
- We can specify stopping sequences to avoid run-on answers.

This quick test ensures everything is hooked up correctly—no errors means we can proceed.


In [ ]:
output = llm(
      "Q: What are you? A: ", # Prompt
      max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n", "``` \n ```"], # Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output)


llama_print_timings:        load time =     613.08 ms
llama_print_timings:      sample time =      31.34 ms /    22 runs   (    1.42 ms per token,   701.91 tokens per second)
llama_print_timings: prompt eval time =     612.99 ms /    10 tokens (   61.30 ms per token,    16.31 tokens per second)
llama_print_timings:        eval time =     740.75 ms /    21 runs   (   35.27 ms per token,    28.35 tokens per second)
llama_print_timings:       total time =    1401.38 ms /    31 tokens


{'id': 'cmpl-574146a3-a071-4c35-99ca-bf41cfa68204', 'object': 'text_completion', 'created': 1738429001, 'model': '/content/notebooks/OpenCoder-8B-Instruct-Q6_K_L.gguf', 'choices': [{'text': "Q: What are you? A:  I'm an artificial intelligence, capable of understanding and generating human language through a complex combination of algorithms and data.", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 10, 'completion_tokens': 22, 'total_tokens': 32}}


### **Clean Up the Output**

We can reduce verbosity by setting

verbose=False

in the Llama constructor. Then we define a ```query_llm()``` function that:

- Define a **system prompt** in the message parameter.
    - A system prompt is a structure text models are trained on that defines how the LLM should respond.  
- Receives a prompt.
- Calls ```llm.create_chat_completion()``` with chosen parameters (like ```max_tokens```, ```top tokens```, etc.).
- Returns a cleaned-up text response.



In [ ]:

llm = Llama(
      model_path=llm_file_path,
      n_gpu_layers=-1, # -1 means all layers offloaded to GPU
      verbose=False,
      n_ctx=2048, # Uncomment to increase the context window
)

def query_llm(prompt):
  output=llm.create_chat_completion(
      max_tokens=256,
      stop=["Q:","\n ```"],
      messages = [
          {"role": "system", "content": "You are an AI that writes very efficient concise python code."},
          {
              "role": "user",
              "content": prompt
              }
      ]
)
  return print(output['choices'][0]['message']['content'])

In [ ]:
query_llm("write a poem about a quick sort algorithm in python.")

In the heart of sorting, QuickSort dwells,
Divide and conquer it is named, with efficiency so high.
First, you pick a pivot point, then you partition,
Items less than pivot go to left, more to right.

The recursion goes deeper, each time narrowing down,
Until at last, your list is sorted in just a few steps.

It's a simple yet powerful algorithm, QuickSort shows,
With its efficiency, it's the go-to choice for many. 

So let's sort some data with QuickSort, shall we?
```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)
```
```
```python
print(quick_sort([3,6,8,10,1,2,1]))
# Output: [1, 1, 2, 3, 6, 8, 10]
```
```
``


### **Tuning Generation Settings**

This is messy and it doesn't stop until it runs into the 256 token limit.

- Repeat Penalty: Increases penalty on repetitive tokens. Helps reduce repetitive or looping answers.
- Temperature: Controls the randomness. Lower temperature => more deterministic output; higher => more creative or varied.
- Stop Sequences: Tells the model when to halt (e.g., ```[STOP]```).

In [ ]:
def query_llm(prompt):
  output=llm.create_chat_completion(
      max_tokens=1024,
      repeat_penalty=1.2,
      temperature=0.7,
      stop=["[STOP]","STOP"],
      messages = [
          {"role": "system", "content": "You are an AI poet of the premier quality that writes very efficient concise poetry about python code. When you are finished, write EXACTLY [STOP] on its own line. No extra punctuation or text."},
          {
              "role": "user",
              "content": prompt
              }
      ]
)
  return print(output['choices'][0]['message']['content'])
query_llm("write a poem about a quick sort algorithm in python.")


Once upon a time there was an algorithm called Quick Sort.
It is known for sorting arrays quickly and efficiently.

Here's how it worked:
1. It chose a 'pivot' element from the array,
2. Partitioned other elements into two halves based on whether they were less or greater than the pivot,
3. And then recursively applied itself to those two sub-arrays, until everything was sorted in order.
  
It had an average and best-case time complexity of O(n log n),
and it performed well even with large data sets.

But like every algorithm, Quick Sort isn't without its downsides:
4. It can be less stable than other sorting algorithms,
5. And worst case performance is O(n^2) if the pivot selection strategy results in unbalanced partitions.
   
Despite these drawbacks, Quick Sort remains a favorite among programmers due to its efficiency and simplicity.

EXACTLY [


### Trailing character '['

This may have generated, let's clean it up. This happens because we have two different 'stop' words.
We are adding ``` cleaned_response = response.rstrip('[') ```

This will start at the end of the string and examine for one example of the character [ and remove it.

In [ ]:
# prompt: function to clean up trailing '[' from the llm return

def query_llm(prompt):
  output=llm.create_chat_completion(
      max_tokens=1024,
      repeat_penalty=1.2,
      temperature=0.7,
      stop=["[STOP]","STOP"],
      messages = [
          {"role": "system", "content": "You are an AI that writes very efficient concise python code. When you are finished, write EXACTLY [STOP] on its own line. No extra punctuation or text."},
          {
              "role": "user",
              "content": prompt
              }
      ]
)
  response = output['choices'][0]['message']['content']
  #Clean up trailing '['
  cleaned_response = response.rstrip('[')
  return print(cleaned_response)

In [ ]:
query_llm("write a poem about a quick sort algorithm in python.")

Once upon a digital heap,
Where sorting algorithms were gathered.

A new friend they named QuickSort,
To sort arrays with speed and charm.

Quick, fast and efficient like lightning,
It's the reigning champion of sorts.

Divide and conquer it taught,
With pivot points dividing lists into halves.

And then each half is sorted independently,
Until eventually all items are in order.

That's how a quicksort algorithm works,
A tale told by those who code well. 

So go forth, brave programmers!
Test your skills with QuickSort.



### **Streaming Responses**

To mimic real-time feedback, we use:

```
stream=True

print(stream["choices"][0]["delta"].get("content", ""), end="")
```

with ```
create_chat_completion()
```. This yields tokens as they’re generated, letting you see partial output in “streaming” style, similar to ChatGPT.

In [ ]:
def stream_query_llm(prompt):
  for stream in llm.create_chat_completion(
      max_tokens=1024,
      stream=True,
      stop=["[STOP]","STOP"],

      messages = [
          {"role": "system", "content": "You are an AI that writes very efficient concise python code. When you are finished, write EXACTLY [STOP] on its own line. No extra punctuation or text."},
          {
              "role": "user",
              "content": prompt
              }
      ],

  ):

    print(stream["choices"][0]["delta"].get("content", ""), end="")
stream_query_llm("write a poem about a quick sort algorithm in python.")


Once upon a time there was a man named Tim,
He loved to sort lists of numbers with Quick Sort.
Quick Sort is fast, it's efficient and easy to use,
It uses divide and conquer strategy for sorting.

Tim started by choosing a pivot point,
Then he partitioned the list around that pivot,
All elements less than the pivot went to its left,
And all greater ones went to its right.

He repeated this process recursively on both sides,
Until each side was sorted in ascending order.
Finally, Tim combined them together and got a sorted list.

This is how Quick Sort works,
It's fast, efficient and easy to understand.
[

### Streaming Trailing Character

The trailing character is back because we are print the "delta" instead of the entire message, so our prior method of ```.rstrip('[')``` wont work.

The model is generating the correct stop tokens, however since we include both ```[STOP] and STOP``` to always catch the stop we need to find that trailing '['. We will check if the token is '[' and break the function if the parameter "finish_reason" is not 'None'. This allows us to only remove a trailing '[' and not '[' used in code we intended to generate.

In [ ]:
import sys

def stream_query_llm(prompt):
    partial_response = ""  # Store the growing response

    for stream in llm.create_chat_completion(
        max_tokens=1024,
        stream=True,
        stop=["[STOP]", "STOP"],  # Stop words
        messages=[
            {"role": "system", "content": "You are an AI that writes very efficient concise python code. When you are finished, write EXACTLY [STOP] on its own line. No extra punctuation or text."},
            {"role": "user", "content": prompt}
        ],
    ):
        token = stream["choices"][0]["delta"].get("content", "")

        # **Skip token if it's exactly "[" (avoiding trailing issues)**
        if token == "[" and stream["choices"][0]["finish_reason"] != 'None':
            continue  # Skip processing this token

                # Append to response
        partial_response += token
        sys.stdout.write(token)  # Print live output
        sys.stdout.flush()




# Example usage
stream_query_llm("Write a poem about a quick sort algorithm in Python.")


Once upon a time there was a man named Quicksort,
He loved to sort data with his fast and efficient method.
Quicksort used a divide-and-conquer strategy,
Dividing the array into two smaller sub-arrays.

The first step was to pick a pivot point,
A value that would help him divide the array.
All values less than the pivot went to the left,
And all greater ones ended up on the right.

This process repeated itself recursively,
Until each sub-array had only one element.
Then, he combined them back together,
Sorting the entire array in no time at all.

Quicksort was a man of many tricks,
With his partition and recursion skills.
He sorted data quickly and efficiently,
And became an overnight sensation.



In [ ]:
stream_query_llm("write a simple function of anykind that includes a list.")


Sure, here is a simple Python function that takes a list as an argument and returns the sum of all elements in the list.

```python
def sum_list(lst):
    return sum(lst)
```

You can use it like this:

```python
numbers = [1, 2, 3, 4, 5]
print(sum_list(numbers))  # Output: 15
```


### **Chat Memory**

We simulate memory by storing all user and assistant messages in a list. Each new prompt includes the entire conversation (or the truncated portion if the context window is large).

- Instantiate ```conversation_memory = []```.
- Each user message and each LLM response are appended as {"role": "user"/"assistant", "content": "..."}.
- Send the entire ```conversation_memory``` to the model each time you want a response.

Context Window Limit:

Remember that each token of conversation consumes part of the LLM’s context window. If your LLM supports an 8K or 6K context window, you can’t exceed it. For longer conversations, you might remove older turns or summarize them.

In [ ]:
# Initialize conversation memory as a list of messages
conversation_memory = []

def memory_query_llm(prompt):
    partial_response = ""  # Store the growing response

    # Append the user's message to the conversation memory
    conversation_memory.append({"role": "user", "content": prompt})

    # Build the messages to include conversation history
    messages = [{"role": "system", "content": "You are a chatty software developer that loves to write poems about code. When you are finished, write EXACTLY [STOP] on its own line. No extra punctuation or text."}] + conversation_memory

    # Stream the LLM's response
    response = ""
    for stream in llm.create_chat_completion(
        max_tokens=1024,
        stream=True,
        stop=["[STOP]"],
        messages=messages,
    ):
        token = stream["choices"][0]["delta"].get("content", "")

        # **Skip token if it's exactly "[" (avoiding trailing issues)**
        if token == "[" and stream["choices"][0]["finish_reason"] != 'None':
            continue  # Skip processing this token

                # Append to response
        partial_response += token
        sys.stdout.write(token)  # Print live output
        sys.stdout.flush()
        response += partial_response

    # Append the AI's response to the conversation memory
    conversation_memory.append({"role": "assistant", "content": response.strip()})

# Example usage
memory_query_llm("Remember my name is Alan, what is my name?")



Your name is Alan.


In [ ]:
memory_query_llm("Write a poem about me using my name, and how I love to develop Generative AI tools")

In the vast expanse of code,
Alan stands as a beacon of brilliance.
His work in artificial intelligence,
Has revolutionized the world we live in.

He codes with precision like a virtuoso,
A machine learning genius at heart.
Every line he writes is not just code,
But a symphony of logic and creativity.

Alan's tools are not just for coding,
They're for understanding, analyzing, and generating.
His Generative AI tools are powerful,
And they make the world a better place to live in.

In the end, Alan stands tall,
A man who codes with passion and pride.


## **Building a Simple Chat Loop**

We can create a simple chat loop in a single cell using ```input()``` in Colab. The user types queries; the model streams responses back. We maintain memory by continually appending messages to the conversation history:

```
def chat_with_llm():
    # ...
    while True:
        user_input = input("You: ").strip()
        # ...
        for stream in llm.create_chat_completion(...):
            # ...
```

This provides an interactive, console-style chat in your Colab cell.



In [ ]:
# Initialize conversation memory
conversation_memory = [
    {"role": "system", "content": "Your role is 'system'. You have a memory and access to prior conversation turns. You take a single turn. When you are finished, write EXACTLY [STOP] on its own line. No extra punctuation or text."}]
def chat_with_llm():
    print("AI Chatbot: Let's chat! Type 'exit' to end the conversation.")
    while True:


        # Get user input
        user_input = input("You: ").strip()

        # Exit condition
        if user_input.lower() == "exit":
            print("AI Chatbot: Goodbye!")
            break

        # Append user input to conversation memory
        conversation_memory.append({"role": "user", "content": user_input})

        # Generate response from the LLM
        response = ""
        for stream in llm.create_chat_completion(
            max_tokens=512,
            stream=True,
            stop=["[STOP]", "STOP"],
            messages=conversation_memory,
        ):
            partial_response = ""  # Store the growing response
            token = stream["choices"][0]["delta"].get("content", "")

            # **Skip token if it's exactly "[" (avoiding trailing issues)**
            if token == "[" and stream["choices"][0]["finish_reason"] != 'None':
                continue  # Skip processing this token

                    # Append to response
            partial_response += token
            sys.stdout.write(token)  # Print live output
            sys.stdout.flush()
            response += partial_response

        print()  # Add a newline for better formatting

        # Append the AI's response to the conversation memory
        conversation_memory.append({"role": "assistant", "content": response.strip()})

# Start the chat
chat_with_llm()


AI Chatbot: Let's chat! Type 'exit' to end the conversation.
You: I am a photographer
As an AI language model, I don't have the capability of understanding context or role-playing in real-time. However, I can provide information based on your input. You mentioned that you are a photographer. What would you like to know about this profession?

You: I have concerns about how my profession has been used in AI
As an AI language model, I don't have personal experiences or opinions. However, I can provide information based on the data and knowledge I've been programmed with. The use of AI in photography is a growing trend, especially in recent years due to advancements in technology. Some professionals are concerned about job displacement, but others see it as an opportunity for growth and innovation. It largely depends on how we utilize this new technology.
AI can assist in various ways such as editing photos, creating personalized photo shoots, analyzing images, etc. However, it's importan

KeyboardInterrupt: Interrupted by user

## **Retrieval-Augmented Generation (RAG)**

RAG is a popular technique to provide an LLM with relevant external information (e.g., from your documents, knowledge base, or PDFs) at query time. This approach ensures the model’s answers are grounded in real data, not just the model’s baked-in knowledge.

Workflow:

- Ingest Text:
    - Split large documents into manageable chunks.
    Create vector embeddings for each chunk (using a model like sentence-transformers).
    Store the chunks and embeddings in a dataframe or specialized vector DB.

- Query:
    - Embed the user’s query with the same embedding model.
    Compute similarity scores (e.g., dot product, cosine similarity) between query embedding and your stored chunk embeddings.
    Retrieve the top-k most relevant chunks.

    - Append these top chunks as context to your LLM prompt.

    - LLM uses both the user query and the retrieved context to produce an answer grounded in your data.

In this notebook:

- We used a precomputed CSV of chunks + embeddings, loaded them, and used sentence-transformers to quickly retrieve top matches for a user query.
Then we built a prompt containing “Document Summary” and “Page Summaries,” appended that to the user query, and sent it to the LLM.



In [ ]:
from huggingface_hub import snapshot_download
import os

embedding_file_name = "BAAI/bge-m3"
embedding_file_path = os.path.join(nb_path, embedding_file_name)

if os.path.isfile(embedding_file_path):
    print(f"'{embedding_file_name}' is already present in '{nb_path}'.")
else:
    print(f"'{embedding_file_name}' not found. Downloading now...")
    embedding_model_path = snapshot_download(
        repo_id="BAAI/bge-m3",

        local_dir=nb_path
    )
    print("Model downloaded to:", embedding_model_path)

### Download Links

Follow the links, download the files, then upload to your notebook.

[Colorado Constitution Embeddings](https://drive.google.com/file/d/1n_EoeAyCQrtKzZMhNxd9PD3p2u68afBN/view?usp=sharing)

[Colorado Constitution Enhanced Output](https://drive.google.com/file/d/1VwNf5IpIeWB3ZYxoSkj__AS-5l_UbfUs/view?usp=sharing)


In [ ]:
# ---------- PATHS --------------
embeddings_df_save_path = "/content/ColoradoConstitution_embeddings.csv"
enhanced_json_file = "/content/ColoradoConstitution_enhanced_output.json"


### Load Embeddings

In [ ]:
# ---------- LOAD JSON --------------
import json
import pandas as pd
import numpy as np

def load_enhanced_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

enhanced_data = load_enhanced_json(enhanced_json_file)
document_summary = enhanced_data.get('document_summary', 'No document summary available.')
page_summaries = {
    int(page): data['summary']
    for page, data in enhanced_data.get('pages', {}).items()
}

# ---------- LOAD CSV + EMBEDDINGS --------------
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)

# Convert the stringified embeddings back to numpy arrays
text_chunks_and_embedding_df_load['embedding'] = text_chunks_and_embedding_df_load['embedding'].apply(
    lambda x: np.array(json.loads(x))
)

pages_and_chunks = text_chunks_and_embedding_df_load.to_dict(orient="records")

In [ ]:
# Convert to torch tensor
import torch
from sentence_transformers import SentenceTransformer, util


embeddings = torch.tensor(
    np.array(text_chunks_and_embedding_df_load["embedding"].tolist()),
    dtype=torch.float32
).to('cpu')

# ---------- LOAD SENTENCE-TRANSFORMER MODEL --------------
embedding_model = SentenceTransformer(model_name_or_path=embedding_model_path, device='cpu')


### Utility Functions

In [ ]:
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

def retrieve_relevant_resources(query: str,
                                embeddings: torch.Tensor,
                                model: SentenceTransformer,
                                n_resources_to_return: int = 4):
    """
    Given the user query, embed it, compute dot-product scores,
    and return top-k matching indices in descending order of relevance.
    """
    query_embedding = model.encode(query, convert_to_tensor=True)
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    scores, indices = torch.topk(dot_scores, k=n_resources_to_return)
    return scores, indices

def get_top_context(query: str,
                    embeddings: torch.Tensor,
                    pages_and_chunks: list[dict],
                    n_resources_to_return: int = 4):
    """
    Fetch the top-k chunks from pages_and_chunks for the user query.
    Returns the chunk dicts in descending order of relevance.
    """
    scores, indices = retrieve_relevant_resources(
        query=query,
        embeddings=embeddings,
        model=embedding_model,
        n_resources_to_return=n_resources_to_return
    )
    # Prepare the context items
    top_context_items = []
    for i, idx in enumerate(indices):
        idx_int = int(idx)  # ensure it's an int
        if 0 <= idx_int < len(pages_and_chunks):
            top_context_items.append(pages_and_chunks[idx_int])
    return top_context_items

def prompt_formatter(query: str, context_items: list[dict]) -> str:
    """
    Build a final prompt to feed into the LLM.
    Includes the document summary and relevant page summaries + chunk text.
    """
    formatted_context = f"Document Summary: {document_summary}\n\n"
    for item in context_items:
        page_number = item.get('page', 'Unknown')
        page_summary = page_summaries.get(page_number, 'No page summary available.')
        content = item.get('content', 'Chunk not found.')
        formatted_context += f"Page {page_number}: {page_summary}\n"
        formatted_context += f"Content: {content}\n\n"

    base_prompt = """Use the following context to answer the user query:
    {context}
User query: {query}
Answer: """
    # For debug:
    # print("PROMPT:", base_prompt.format(context=formatted_context, query=query))
    return base_prompt.format(context=formatted_context, query=query)




### Clear GPU VRAM

In [ ]:
# This is critical or else we will load the model again and run out of memory.

# Import Garbage Collector, Python has a quirk where it does not release memory if it might still be called again.
import gc

# Delete the variable llm then force garbage collections and clear the GPU VRAM
del llm
gc.collect()
torch.cuda.empty_cache()


### **Remount the Model with an expanded context window**

Reinitialize our model with an expanded context window:

```n_ctx=8000```

This allows for more text in the input, so we can feed in more user messages or more retrieved text.

In [ ]:


from llama_cpp import Llama

llm = Llama(
      model_path=llm_file_path,
      n_gpu_layers=-1, # -1 means all layers offloaded to GPU
      verbose=False,
      n_ctx=8000,
)

### Run as a Chat Loop

In [ ]:


def chat_loop():
    print("Welcome to the *Colorado Constitution* QA Chat! (Type 'exit' to quit)")

    while True:
        # Prompt the user
        user_query = input("\nUser: ").strip()

        # Exit on demand
        if user_query.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        # 1. Retrieve top context
        top_context = get_top_context(query=user_query,
                                      embeddings=embeddings,
                                      pages_and_chunks=pages_and_chunks,
                                      n_resources_to_return=10)

        # 2. Format the prompt
        prompt = prompt_formatter(query=user_query, context_items=top_context)
        system_prompt = [
  {
    "role": "system",
    "content": "You are a concise and helpful system. You will receive context along with associated page numbers. Begin each answer with a line in the format 'PAGE [page_number]:' referencing the relevant page(s). Then provide a concise summary of the answer to the user's query, strictly avoiding quoting exact text from the context. When you finish, write EXACTLY 'STOP' on a separate line—no extra punctuation or text."
  }
]


        system_prompt.append({"role":"user","content":prompt})
        # DEBUG
        # print(system_prompt)

        # Generate response from the LLM
        response = ""
        for stream in llm.create_chat_completion(
            max_tokens=1000,
            stream=True,
            stop=["STOP"],
            messages=system_prompt,
        ):
            content = stream["choices"][0]["delta"].get("content", "")
            print(content, end="")
            # response += content

        print()  # Add a newline for better formatting






### Questions to Ask
- What are the rights of Colorado citizens that go beyond the U.S. Constitution?

- How can a citizen propose a new law or constitutional amendment in Colorado?"

## **Function Calling**

Some LLM frameworks (like OpenAI’s GPT-4 or adapted llama.cpp builds) are trained on specialized JSON “function calls.” The idea is:

- User prompt triggers a “function call” in the model’s output.
- The model returns a structured JSON object naming the function and its arguments.
- Your code parses that output, calls the actual function in Python, and returns results.

We define:

```
def ChatResponse(text: str) -> dict:
# Handle standard chat functions
  ```

```
def UserDetail(name: str, age: int) -> dict:
 # Store or retrieve user details
 ```

We then map function names to Python functions using ```FUNCTION_MAP``` and parse the model’s output to see if it invoked ```UserDetail() or ChatResponse()```.


In [ ]:
def UserDetail(name: str, age: int) -> dict:
    """
    Stores or retrieves user details.

    If the user is new, their details are added.
    If the user already exists, their details are retrieved.
    """

    # Check if the user is already stored
    if name in users:
        return {"message": f"User {name} already exists.", "data": users[name]}

    # Store new user details
    users[name] = {"name": name, "age": age}
    return f"""User {name} added successfully.", "data": {users[name]}"""

In [ ]:
def ChatResponse(text: str) -> dict:

    """
    Handles standard chat responses.
    Introducing the function format confuses the model on normal chat responses.
    """
    output=llm.create_chat_completion(
      max_tokens=1024,
      repeat_penalty=1.2,
      temperature=0.7,
      stop=["[STOP]","STOP"],
      messages = [
          {"role": "system", "content": "You are an AI chatting with a user. When you are finished, write EXACTLY [STOP] on its own line. No extra punctuation or text."},
          {
              "role": "user",
              "content": text
              }
      ]
)
    return text

### Robustly clean up the response string

This model is not explicitly trained for function calling, and requires support. The string returned is useful JSON that requires some cleaning.

This function uses the regular expression library to find line breaks and trailing text or spaces and removes them.

This is ultimately very fragile and dependent on the model to output valid JSON or consistently invalid JSON in ways we can detect and fix.

In [ ]:
import json
import re

# Global dictionary to store user details
users = {}

# A mapping of function names to actual Python callables
FUNCTION_MAP = {
    "UserDetail": UserDetail,
    "ChatResponse": ChatResponse
}

import re


import re
import json

def clean_json_string(json_str: str) -> str:
    """
    Cleans up malformed JSON output from an LLM function call response.
    - Ensures proper closing of quotes and brackets.
    - Fixes truncated JSON issues.
    - Removes trailing characters after JSON.
    """
    if not isinstance(json_str, str):
        return json_str  # Ensure it's a string before processing

    json_str = json_str.strip()  # Remove leading/trailing spaces

    # 🔹 Detect missing closing quote in "text" fields
    quote_count = json_str.count('"')
    if quote_count % 2 != 0:  # Odd number of quotes → missing closing quote
        json_str += '"'  # Add missing quote to close the string

    # 🔹 Fix unbalanced brackets `{}` or `[]`
    open_braces = json_str.count("{")
    close_braces = json_str.count("}")
    if open_braces > close_braces:
        json_str += "}" * (open_braces - close_braces)  # Add missing closing braces
    elif close_braces > open_braces:
        json_str = json_str.rstrip("}")  # Remove excess closing braces

    open_brackets = json_str.count("[")
    close_brackets = json_str.count("]")
    if open_brackets > close_brackets:
        json_str += "]" * (open_brackets - close_brackets)
    elif close_brackets > open_brackets:
        json_str = json_str.rstrip("]")

    # 🔹 Remove trailing junk after JSON structure
    json_str = re.sub(r'}[^\}]*$', '}', json_str)  # Removes extra non-JSON text

    # 🔹 Validate the fixed JSON
    try:
        parsed_json = json.loads(json_str)  # Ensure it's valid JSON
        return json.dumps(parsed_json)  # Return cleaned JSON as a string
    except json.JSONDecodeError:
        return "Error: Unable to fully clean JSON."



### Handle the function call

This function first checks for the Key "function_call" and if present extracts it and executes the function as a standard python function. It has a fallback to clean the string up if it is present but cannot be treated as JSON.


In [ ]:
def handle_llama_function_call(response: dict, function_map: dict):
    """
    Parses a response and determines if it's a function call or a normal chat response.
    Robust Error catching.
    Introducing 'Function Calls' has many opportunities to fail.
    """
    try:
        llama_message = response["choices"][0]["message"]
        print(llama_message)
    except (KeyError, IndexError):
        return "Error: No valid message found in response."

    function_call = llama_message.get("function_call")

    if not function_call:
        print("No function_call detected. Cleaning content...")

        if "content" not in llama_message:
            return "Error: No 'content' field found in response."

        # Clean the content string before parsing
        cleaned_content = clean_json_string(llama_message["content"])
        print(f"Cleaned JSON String: {cleaned_content}")

        # Try to parse the cleaned content
        try:
            function_call_wrapper = json.loads(cleaned_content)
            print(f"Parsed function_call: {function_call_wrapper}")
                    # Ensure function_call is a dictionary before proceeding
            if not isinstance(function_call_wrapper, dict):
                return f"Error: Expected function_call to be a dictionary, but got {type(function_call_wrapper)}."
                    # get map of function_call
            function_call = function_call_wrapper.get("function_call")
            if not function_call:
                return "Error: No function_call found in function_call_wrapper."
        except json.JSONDecodeError:
            return f"Error: Unable to parse function call: {cleaned_content}"

    function_name = function_call.get("name")
    # print(f"Function Name: {function_name}")

    if not function_name:
        return "Error: No function name found in function_call."
    args_str = function_call.get("arguments", "{}")

    # Ensure arguments are parsed correctly (handles nested JSON issue)
    try:
        function_args = json.loads(args_str) if isinstance(args_str, str) else args_str
    except json.JSONDecodeError:
        return f"Error: Unable to parse function arguments: {args_str}"

    func = function_map.get(function_name)
    if not func:
        return f"Error: No matching function for '{function_name}'"

    # Execute the function
    result = func(**function_args)
    return result


### Test each function call and Prompt

Here we are using the llama.cpp built in tool call framework to add tools to the prompt passed to the llm. This is based on the OpenAI function call standard, which is fairly universal.


In [ ]:
print(f"Users Variable before: {users}")
test_function_call = llm.create_chat_completion(
      messages = [
        {
          "role": "system",
          "content": "You are a function calling chatbot that responds in standard JSON format. The assistant gives helpful, detailed, and polite answers to the user's questions. The assistant calls functions with appropriate input when necessary"

        },
        {
          "role": "user",
          "content": "Extract Jason is 25 years old"
        }
      ],
      tools=[{
        "type": "function",
        "function": {
          "name": "UserDetail",
          "parameters": {
            "type": "object",
            "title": "UserDetail",
            "properties": {
              "name": {
                "title": "name",
                "type": "string"
              },
              "age": {
                "title": "age",
                "type": "integer"
              }
            },
            "required": [ "name", "age" ]
          }
        }
      }],
      tool_choice={
        "type": "function",
        "function": {
          "name": "UserDetail"
        }
      }
)

print(test_function_call)
handle_llama_function_call(test_function_call, FUNCTION_MAP)
print(f"Users Variable after: {users}")

Users Variable before: {}
{'id': 'chatcmpl-8b158ee7-5dfd-44de-8073-ccd223ddb3a2', 'object': 'chat.completion', 'created': 1738430580, 'model': '/content/notebooks/OpenCoder-8B-Instruct-Q6_K_L.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'UserDetail', 'arguments': '{"name": "ExtractJason", "age": 25} '}, 'tool_calls': [{'id': 'call__0_UserDetail_cmpl-8b158ee7-5dfd-44de-8073-ccd223ddb3a2', 'type': 'function', 'function': {'name': 'UserDetail', 'arguments': '{"name": "ExtractJason", "age": 25} '}}]}, 'logprobs': None, 'finish_reason': 'tool_calls'}], 'usage': {'prompt_tokens': 61, 'completion_tokens': 14, 'total_tokens': 75}}
{'role': 'assistant', 'content': None, 'function_call': {'name': 'UserDetail', 'arguments': '{"name": "ExtractJason", "age": 25} '}, 'tool_calls': [{'id': 'call__0_UserDetail_cmpl-8b158ee7-5dfd-44de-8073-ccd223ddb3a2', 'type': 'function', 'function': {'name': 'UserDetail', 'arguments': '{"name": "ExtractJ

In [ ]:
test_function_call = llm.create_chat_completion(
      messages = [
        {
          "role": "system",
          "content": "You are a function calling chatbot that responds in standard JSON format. The assistant gives helpful, detailed, and polite answers to the user's questions. The assistant calls functions with appropriate input when necessary"

        },
        {
          "role": "user",
          "content": "Respond to Hello?"
        }
      ],
      tools=[{
      "type": "function",
      "function": {
        "name": "ChatResponse",

        "parameters": {
          "type": "object",
          "properties": {
            "text": {
              "type": "string",

            }
          },
          "required": ["text"]
        }
      }
    }],
      tool_choice={
        "type": "function",
        "function": {
          "name": "ChatResponse"
        }
      }
)

handle_llama_function_call(test_function_call, FUNCTION_MAP)

{'role': 'assistant', 'content': None, 'function_call': {'name': 'ChatResponse', 'arguments': '{"text": "Hello! How can I assist you today?"} '}, 'tool_calls': [{'id': 'call__0_ChatResponse_cmpl-6c576baf-2cc5-4aea-80d8-b93bbb1f35f1', 'type': 'function', 'function': {'name': 'ChatResponse', 'arguments': '{"text": "Hello! How can I assist you today?"} '}}]}


'Hello! How can I assist you today?'

### Rebuild the tools list to include both tools

Each tool worked seperately. Now we test if the LLM can make a "decision" on which tool to use and respond appropriately. To do this we need to provide it with both tools

In [ ]:

tool_choice = [
    {
        "type": "function",
        "function": {
            "name": "ChatResponse"
        }
    },
    {
        "type": "function",
        "function": {
            "name": "UserDetail"
        }
    }
]


### Rework the system prompt

We need to write a very explicit and clear prompt to guide the LLM on what to do and what the tools provided do. This prompt was generating uses a technique called "Meta-prompting". I wrote the initial prompt which got about 80% accuracy from the llm. Then I asked GPT-o1 to rewrite and clarify the prompt which is provided here. With a little further refinement we now have a *mostly accurate function call prompt.



In [ ]:
system_prompt = """You are a function call system that ALWAYS responds with a VALID JSON object.
 **RULE: The JSON MUST ALWAYS BE PROPERLY CLOSED with a SINGLE `}` at the end.**

Your response MUST follow these rules:

1. **No Disclaimers**: Do NOT include phrases like "As an AI" or "I am a Machine Learning model." Focus ONLY on the user's request.

2. **STRICT JSON-Only Mode**: NEVER return plain text outside JSON.
   - Your entire response must be a **single valid JSON object**.
   - Example: { "function_call": { "name": "FUNCTION_NAME", "arguments": { "KEY_1": "value1", "KEY_2": "value2" } } }

3. **Key Requirements**:
- The **ONLY** top-level key is `"function_call"`. No other keys allowed.
- Inside `"function_call"`, include:
  - `"name"`: Either `"UserDetail"` or `"ChatResponse"`.
  - `"arguments"`: A **valid JSON object** with the required fields.

4. **Function Behavior**:
- If the user enters `"ADD"` followed by a **name and age**, call:
{ "function_call": { "name": "UserDetail", "arguments": { "name": "<name>", "age": <integer> } } }

- For ALL other inputs, return:
{ "function_call": { "name": "ChatResponse", "arguments": { "text": "<your response>" } } }

5. **STRICT JSON Formatting**:
- NEVER return **duplicate** `"function_call"` keys.
- NEVER add **extra fields** like `"role"` or `"content"`.
- Your response must be a **fully closed JSON object**.

6. **Line Breaks**:
- All line breaks in `"text"` **MUST** be escaped as `"\\n"`.
- ✅ Correct: `"text": "Hello\\nWorld"`
- ❌ Incorrect: `"text": "Hello
World"`

7. **Ensure the JSON Closes Properly**:
- Your response MUST always **end with exactly one `}`**.
- If the response is too long, TRUNCATE at a full sentence, NEVER mid-word or mid-key.

8. **STOP Token**:
- When you finish, **append `[STOP]` at the end of `"text"` or `"arguments"`**.
- Example:
{ "function_call": { "name": "ChatResponse", "arguments": { "text": "Hello!\nWhat can I do for you?[STOP]" } } }

🚨 **WARNING:** Failing to follow these rules will break system function calls. ONLY valid JSON is allowed."""



### Call the LLM

The actual call to the LLM is nearly identical to the prior functions. We lower the temperature some and pass the list tool_choice.

In [ ]:


def do_function_calls(user_input):
    response = llm.create_chat_completion(
        max_tokens=1024,
        repeat_penalty=1.2,
      temperature=0.4,
      stop=["[STOP]"],
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        tool_choice= tool_choice

        # The list of both function schemas

    )
    return handle_llama_function_call(response, FUNCTION_MAP)


In [ ]:
do_function_calls("ADD Alan is 39 years old")

{'role': 'assistant', 'content': '{ "function_call": { "name": "UserDetail", "arguments": { "name": "Alan", "age": 39 } } }\n'}
No function_call detected. Cleaning content...
Cleaned JSON String: {"function_call": {"name": "UserDetail", "arguments": {"name": "Alan", "age": 39}}}
Parsed function_call: {'function_call': {'name': 'UserDetail', 'arguments': {'name': 'Alan', 'age': 39}}}


'User Alan added successfully.", "data": {\'name\': \'Alan\', \'age\': 39}'

In [ ]:
do_function_calls("Hello! What functions do you have?")

{'role': 'assistant', 'content': '{ "function_call": { "name": "ChatResponse", "arguments": { "text": "I currently support the following functionalities: UserDetail, ChatResponse.'}
No function_call detected. Cleaning content...
Cleaned JSON String: {"function_call": {"name": "ChatResponse", "arguments": {"text": "I currently support the following functionalities: UserDetail, ChatResponse."}}}
Parsed function_call: {'function_call': {'name': 'ChatResponse', 'arguments': {'text': 'I currently support the following functionalities: UserDetail, ChatResponse.'}}}


'I currently support the following functionalities: UserDetail, ChatResponse.'

### Function Call Loop



In [ ]:
def function_chat_loop():
    print("Welcome to the *User Name and Age* Data Storage Tool! (Type 'exit' to quit)")
    print("Type in a name and age and it will be stored")

    while True:
        # Prompt the user
        user_query = input("\nUser: ").strip()

        # Exit on demand
        if user_query.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        response = do_function_calls(user_query)

        print()  # Add a newline for better formatting
        print(response)


In [ ]:
# Finally, call the function:
function_chat_loop()

Welcome to the *User Name and Age* Data Storage Tool! (Type 'exit' to quit)
Type in a name and age and it will be stored

User: ADD Stefanie Age 35
{'role': 'assistant', 'content': '{ "function_call": { "name": "UserDetail", "arguments": { "name": "Stefanie", "age": 35 } } }\n'}
No function_call detected. Cleaning content...
Cleaned JSON String: {"function_call": {"name": "UserDetail", "arguments": {"name": "Stefanie", "age": 35}}}
Parsed function_call: {'function_call': {'name': 'UserDetail', 'arguments': {'name': 'Stefanie', 'age': 35}}}

User Stefanie added successfully.", "data": {'name': 'Stefanie', 'age': 35}


In [ ]:
print(users)


# What We've Learned

- Understand how to prepare a Colab environment for LLM deployment.
- See how to load and run a quantized model (in GGUF format) locally.
- Explore chat features like streaming responses and memory.
- Get a quick introduction to Retrieval-Augmented Generation (RAG) workflows.
- Take a look at function calling concepts with open-source LLMs.

# What to Try Next?

**1. Experiment With Different System Messages**

    Why: The system message heavily influences the LLM’s behavior (tone, style, etc.).
    How: Try adjusting style:
    - Casual Buddy: “You are a friendly assistant who jokes around with the user.”
    - Strict Instructor: “You are a concise tutor who never uses more than 20 words per sentence.”
    - What to Watch For:
        Compare how the same user query yields different responses depending on the system message.

**2. Add More Function Calls**

    Idea: Expand your function map beyond UserDetail and ChatResponse.
    Potential New Functions:
    - Weather Inquiry: get_weather(city: str, date: Optional[str]) -> dict
        - The model can output real-time weather calls (you’d stub or integrate a real API).
    - Calculator: calculate(expression: str) -> float
        - The model can produce math solutions with an external function.
    - Database Lookup: lookup_user_profile(username: str) -> dict
        - The model can retrieve user data from a local dictionary or DB.
    
    Benefit: This transforms your chatbot into a multi-tool system capable of performing real actions based on user prompts.

**3. Try Different Quantized Models or Formats**

    Reason: Not all quantized models behave the same; each has its own performance vs. accuracy trade-offs.
    
    Examples:
    - 7B vs. 13B vs. 30B LLaMA or OpenCoder variants.
    - GGUF vs. GGML vs. CPU-based quantizations.

    What to Compare:
      - Generation speed (tokens/s).
      - Response coherence/quality.
      - GPU/CPU memory usage.

**4. Integrate Image Generation or Multimodal Models**

    Approach: Call a function that taps into a text-to-image model (e.g., Stable Diffusion, SDXL).
    
**5. Play With Other Modalities**

    - Audio: Convert user queries to text via speech-to-text (e.g., OpenAI - - - Whisper or any open-source alternative) and feed them to your chatbot.
    - Video: Basic video summarization or caption generation. You’d pass frames or transcripts to the LLM for summary.
    - Custom Data: Train or fine-tune an LLM on your own dataset (like specialized documents), then integrate that specialized knowledge.

**6. Enhance Retrieval-Augmented Generation (RAG)**

    Why: Make your bot more knowledgeable about specific domains (e.g., legal, academic, or game lore).
    How:
    - Expand your corpus beyond the current dataset (Colorado Constitution, etc.).
    - Use a vector database (like Milvus or FAISS) for faster, scalable retrieval.
    - Incorporate and explore Knowledge Graphs
    - Experiment with chunk sizes and embedding models for better relevance.

**7. Join an Open Model Project**

  The goal of this repo is to build the missing pieces of the R1 pipeline such that everybody can reproduce and build on top of it. The project is simple by design and mostly consists of:

  https://github.com/huggingface/open-r1

# **Resources Used**

Note: These open-source tools are continually evolving, and new quantization formats or faster inference backends are appearing regularly. Keep an eye on the llama.cpp project and the broader HF ecosystem to stay updated.


Model : [OpenCoder-8B-Instruct-Q5_K_L ](https://huggingface.co/infly/OpenCoder-8B-Instruct)

llama.cpp : [llama-cpp-python bindings](https://github.com/abetlen/llama-cpp-python)

Embedding Model : [BAAI bge-m3](https://huggingface.co/BAAI/bge-m3)

Hugging Face Hub : [Huggingface.co](https://huggingface.co/)



# Vocabulary

##General AI & Machine Learning Terms
1. **Model**

    Definition: A trained machine learning system that takes an input (like a user’s message) and generates an output (like a chatbot’s response).
    Example: GPT-4, Llama 2, OpenCoder-8B.
    
2. **Training**

    Definition: The process of teaching an AI model by exposing it to large amounts of text data so it can learn patterns, grammar, and reasoning.

3. **Fine-Tuning**

    Definition: Taking a pre-trained model and refining it further on a specific dataset to specialize in certain tasks.
   
4. **Inference**

    Definition: The process of making predictions or generating responses based on a trained model.
   
## Language Model-Specific Terms
5. **Token**

    Definition: The smallest unit of text a model processes. A token can be a word, subword, or even a single character.
    Example:
        "Hello" = 1 token
        "Artificial Intelligence" = 2 tokens
        "AI" = 1 token

6. **Tokenization**

    Definition: The process of breaking text into tokens so an AI model can process it.
    Example: "Hello, world!" → ["Hello", ",", "world", "!"]

7. **Context Window**

    Definition: The maximum number of tokens a model can process in one request.
    Example: If a model has a 4096-token context window, it can "remember" only the last 4096 tokens when generating a response.
    Impact: If your conversation is too long, the model forgets earlier parts.

8. **Embeddings**

    Definition: Numerical representations of words or sentences that capture meaning in a way that machines understand.
    Example: The words "dog" and "puppy" will have similar embeddings because they have similar meanings.

9. **Retrieval-Augmented Generation (RAG)**

    Definition: A technique where a chatbot retrieves relevant external documents before generating a response.
   
10. **Prompt**

    Definition: The input text given to a model to generate a response.
    
11. **System Prompt**

    Definition: A special instruction that guides the behavior of an AI model before any user input.

12. **Temperature**

    Definition: A setting that controls how random or creative the model's responses are.
    Example:
        Low (0.1): Precise and deterministic.
        High (1.0): More random and creative.

13. **Stop Sequence**

    Definition: A predefined sequence of characters that tells the model when to stop generating text.
  
## Hardware & Performance Terms
14. **Quantization**

    Definition: The process of compressing an AI model to run efficiently on lower-powered hardware. Shortening the coordinate of the vector from 32bit float -> 16, 8, or less bits.
    
15. **GPU Acceleration**

    Definition: Using a Graphics Processing Unit (GPU) to speed up AI computations.
  
16. **VRAM (Video RAM)**

    Definition: The memory available on a GPU, affecting how large an AI model you can load.
   



# Advanced Vocabulary
1. **Vector**
A vector is just a list of numbers that represent something in a way a computer can understand.

Think of a vector as:

    A shopping list with quantities: [2 apples, 3 bananas, 1 orange]
    A GPS location with coordinates: [latitude, longitude]
    A movie’s characteristics: [Action: 8, Romance: 2, Comedy: 5]

Why Are Vectors Useful?

Vectors help computers store and compare information efficiently.
For example, if we represent words as vectors, we can find words with similar meanings by comparing their vectors!
Visualizing Vectors

Imagine a vector as an arrow in space:

    Short arrows mean small numbers (small impact).
    Long arrows mean big numbers (strong impact).
    Direction matters—two arrows pointing the same way are more similar than arrows pointing in opposite directions.

2. **Dot Product**
The dot product is a way to measure how similar two lists of numbers (vectors) are.

Imagine each vector as a list of features, like:

    A person's height, weight, and age.
    A movie's rating on different aspects (acting, story, visuals).
    A word's relationship to different meanings.

How It Works:

To calculate the dot product, you:

    Multiply each pair of numbers in the two lists.
    Add up all those multiplied values.

Example Calculation:

If you have two vectors:

A = [2, 3, 4]  
B = [1, 5, 2]

The dot product is:

(2×1) + (3×5) + (4×2) = 2 + 15 + 8 = 25

Why It’s Useful?

    If the dot product is large, the two vectors are similar.
    If the dot product is zero, the vectors are unrelated.
    If the dot product is negative, they are opposite in some way.
3. **Nearest Neighbor**

Imagine you're at a party and you want to find the person who looks the most like your friend. You scan the room and compare faces—whoever looks most similar is your nearest neighbor.

In AI and machine learning, nearest neighbor means:
👉 Finding the most similar item based on a mathematical comparison.

💡 Example:

    In a recommendation system, if you like a certain movie, the system finds movies with the most similar features (genre, actors, rating, etc.).
    In image search, if you upload a picture of a dog, the system finds pictures with the most similar patterns of colors and shapes.

4. **Top-K**

"Top-K" is just a fancy way of saying "give me the best K matches."

    "Top-3 pizza places" → The 3 highest-rated pizza places.
    "Top-5 nearest neighbors" → The 5 most similar items.

💡 Example:
If you search for a product, an online store might show you the Top-5 recommendations that match what you're looking for.

# Licensed under the GPLv3

  This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program. If not, see <https://www.gnu.org/licenses/>.

                    GNU GENERAL PUBLIC LICENSE
                       Version 3, 29 June 2007

 Copyright (C) 2007 Free Software Foundation, Inc. <https://fsf.org/>
 Everyone is permitted to copy and distribute verbatim copies
 of this license document, but changing it is not allowed.

                            Preamble

  The GNU General Public License is a free, copyleft license for
software and other kinds of works.

  The licenses for most software and other practical works are designed
to take away your freedom to share and change the works.  By contrast,
the GNU General Public License is intended to guarantee your freedom to
share and change all versions of a program--to make sure it remains free
software for all its users.  We, the Free Software Foundation, use the
GNU General Public License for most of our software; it applies also to
any other work released this way by its authors.  You can apply it to
your programs, too.

  When we speak of free software, we are referring to freedom, not
price.  Our General Public Licenses are designed to make sure that you
have the freedom to distribute copies of free software (and charge for
them if you wish), that you receive source code or can get it if you
want it, that you can change the software or use pieces of it in new
free programs, and that you know you can do these things.

  To protect your rights, we need to prevent others from denying you
these rights or asking you to surrender the rights.  Therefore, you have
certain responsibilities if you distribute copies of the software, or if
you modify it: responsibilities to respect the freedom of others.

  For example, if you distribute copies of such a program, whether
gratis or for a fee, you must pass on to the recipients the same
freedoms that you received.  You must make sure that they, too, receive
or can get the source code.  And you must show them these terms so they
know their rights.

  Developers that use the GNU GPL protect your rights with two steps:
(1) assert copyright on the software, and (2) offer you this License
giving you legal permission to copy, distribute and/or modify it.

  For the developers' and authors' protection, the GPL clearly explains
that there is no warranty for this free software.  For both users' and
authors' sake, the GPL requires that modified versions be marked as
changed, so that their problems will not be attributed erroneously to
authors of previous versions.

  Some devices are designed to deny users access to install or run
modified versions of the software inside them, although the manufacturer
can do so.  This is fundamentally incompatible with the aim of
protecting users' freedom to change the software.  The systematic
pattern of such abuse occurs in the area of products for individuals to
use, which is precisely where it is most unacceptable.  Therefore, we
have designed this version of the GPL to prohibit the practice for those
products.  If such problems arise substantially in other domains, we
stand ready to extend this provision to those domains in future versions
of the GPL, as needed to protect the freedom of users.

  Finally, every program is threatened constantly by software patents.
States should not allow patents to restrict development and use of
software on general-purpose computers, but in those that do, we wish to
avoid the special danger that patents applied to a free program could
make it effectively proprietary.  To prevent this, the GPL assures that
patents cannot be used to render the program non-free.

  The precise terms and conditions for copying, distribution and
modification follow.

                       TERMS AND CONDITIONS

  0. Definitions.

  "This License" refers to version 3 of the GNU General Public License.

  "Copyright" also means copyright-like laws that apply to other kinds of
works, such as semiconductor masks.

  "The Program" refers to any copyrightable work licensed under this
License.  Each licensee is addressed as "you".  "Licensees" and
"recipients" may be individuals or organizations.

  To "modify" a work means to copy from or adapt all or part of the work
in a fashion requiring copyright permission, other than the making of an
exact copy.  The resulting work is called a "modified version" of the
earlier work or a work "based on" the earlier work.

  A "covered work" means either the unmodified Program or a work based
on the Program.

  To "propagate" a work means to do anything with it that, without
permission, would make you directly or secondarily liable for
infringement under applicable copyright law, except executing it on a
computer or modifying a private copy.  Propagation includes copying,
distribution (with or without modification), making available to the
public, and in some countries other activities as well.

  To "convey" a work means any kind of propagation that enables other
parties to make or receive copies.  Mere interaction with a user through
a computer network, with no transfer of a copy, is not conveying.

  An interactive user interface displays "Appropriate Legal Notices"
to the extent that it includes a convenient and prominently visible
feature that (1) displays an appropriate copyright notice, and (2)
tells the user that there is no warranty for the work (except to the
extent that warranties are provided), that licensees may convey the
work under this License, and how to view a copy of this License.  If
the interface presents a list of user commands or options, such as a
menu, a prominent item in the list meets this criterion.

  1. Source Code.

  The "source code" for a work means the preferred form of the work
for making modifications to it.  "Object code" means any non-source
form of a work.

  A "Standard Interface" means an interface that either is an official
standard defined by a recognized standards body, or, in the case of
interfaces specified for a particular programming language, one that
is widely used among developers working in that language.

  The "System Libraries" of an executable work include anything, other
than the work as a whole, that (a) is included in the normal form of
packaging a Major Component, but which is not part of that Major
Component, and (b) serves only to enable use of the work with that
Major Component, or to implement a Standard Interface for which an
implementation is available to the public in source code form.  A
"Major Component", in this context, means a major essential component
(kernel, window system, and so on) of the specific operating system
(if any) on which the executable work runs, or a compiler used to
produce the work, or an object code interpreter used to run it.

  The "Corresponding Source" for a work in object code form means all
the source code needed to generate, install, and (for an executable
work) run the object code and to modify the work, including scripts to
control those activities.  However, it does not include the work's
System Libraries, or general-purpose tools or generally available free
programs which are used unmodified in performing those activities but
which are not part of the work.  For example, Corresponding Source
includes interface definition files associated with source files for
the work, and the source code for shared libraries and dynamically
linked subprograms that the work is specifically designed to require,
such as by intimate data communication or control flow between those
subprograms and other parts of the work.

  The Corresponding Source need not include anything that users
can regenerate automatically from other parts of the Corresponding
Source.

  The Corresponding Source for a work in source code form is that
same work.

  2. Basic Permissions.

  All rights granted under this License are granted for the term of
copyright on the Program, and are irrevocable provided the stated
conditions are met.  This License explicitly affirms your unlimited
permission to run the unmodified Program.  The output from running a
covered work is covered by this License only if the output, given its
content, constitutes a covered work.  This License acknowledges your
rights of fair use or other equivalent, as provided by copyright law.

  You may make, run and propagate covered works that you do not
convey, without conditions so long as your license otherwise remains
in force.  You may convey covered works to others for the sole purpose
of having them make modifications exclusively for you, or provide you
with facilities for running those works, provided that you comply with
the terms of this License in conveying all material for which you do
not control copyright.  Those thus making or running the covered works
for you must do so exclusively on your behalf, under your direction
and control, on terms that prohibit them from making any copies of
your copyrighted material outside their relationship with you.

  Conveying under any other circumstances is permitted solely under
the conditions stated below.  Sublicensing is not allowed; section 10
makes it unnecessary.

  3. Protecting Users' Legal Rights From Anti-Circumvention Law.

  No covered work shall be deemed part of an effective technological
measure under any applicable law fulfilling obligations under article
11 of the WIPO copyright treaty adopted on 20 December 1996, or
similar laws prohibiting or restricting circumvention of such
measures.

  When you convey a covered work, you waive any legal power to forbid
circumvention of technological measures to the extent such circumvention
is effected by exercising rights under this License with respect to
the covered work, and you disclaim any intention to limit operation or
modification of the work as a means of enforcing, against the work's
users, your or third parties' legal rights to forbid circumvention of
technological measures.

  4. Conveying Verbatim Copies.

  You may convey verbatim copies of the Program's source code as you
receive it, in any medium, provided that you conspicuously and
appropriately publish on each copy an appropriate copyright notice;
keep intact all notices stating that this License and any
non-permissive terms added in accord with section 7 apply to the code;
keep intact all notices of the absence of any warranty; and give all
recipients a copy of this License along with the Program.

  You may charge any price or no price for each copy that you convey,
and you may offer support or warranty protection for a fee.

  5. Conveying Modified Source Versions.

  You may convey a work based on the Program, or the modifications to
produce it from the Program, in the form of source code under the
terms of section 4, provided that you also meet all of these conditions:

    a) The work must carry prominent notices stating that you modified
    it, and giving a relevant date.

    b) The work must carry prominent notices stating that it is
    released under this License and any conditions added under section
    7.  This requirement modifies the requirement in section 4 to
    "keep intact all notices".

    c) You must license the entire work, as a whole, under this
    License to anyone who comes into possession of a copy.  This
    License will therefore apply, along with any applicable section 7
    additional terms, to the whole of the work, and all its parts,
    regardless of how they are packaged.  This License gives no
    permission to license the work in any other way, but it does not
    invalidate such permission if you have separately received it.

    d) If the work has interactive user interfaces, each must display
    Appropriate Legal Notices; however, if the Program has interactive
    interfaces that do not display Appropriate Legal Notices, your
    work need not make them do so.

  A compilation of a covered work with other separate and independent
works, which are not by their nature extensions of the covered work,
and which are not combined with it such as to form a larger program,
in or on a volume of a storage or distribution medium, is called an
"aggregate" if the compilation and its resulting copyright are not
used to limit the access or legal rights of the compilation's users
beyond what the individual works permit.  Inclusion of a covered work
in an aggregate does not cause this License to apply to the other
parts of the aggregate.

  6. Conveying Non-Source Forms.

  You may convey a covered work in object code form under the terms
of sections 4 and 5, provided that you also convey the
machine-readable Corresponding Source under the terms of this License,
in one of these ways:

    a) Convey the object code in, or embodied in, a physical product
    (including a physical distribution medium), accompanied by the
    Corresponding Source fixed on a durable physical medium
    customarily used for software interchange.

    b) Convey the object code in, or embodied in, a physical product
    (including a physical distribution medium), accompanied by a
    written offer, valid for at least three years and valid for as
    long as you offer spare parts or customer support for that product
    model, to give anyone who possesses the object code either (1) a
    copy of the Corresponding Source for all the software in the
    product that is covered by this License, on a durable physical
    medium customarily used for software interchange, for a price no
    more than your reasonable cost of physically performing this
    conveying of source, or (2) access to copy the
    Corresponding Source from a network server at no charge.

    c) Convey individual copies of the object code with a copy of the
    written offer to provide the Corresponding Source.  This
    alternative is allowed only occasionally and noncommercially, and
    only if you received the object code with such an offer, in accord
    with subsection 6b.

    d) Convey the object code by offering access from a designated
    place (gratis or for a charge), and offer equivalent access to the
    Corresponding Source in the same way through the same place at no
    further charge.  You need not require recipients to copy the
    Corresponding Source along with the object code.  If the place to
    copy the object code is a network server, the Corresponding Source
    may be on a different server (operated by you or a third party)
    that supports equivalent copying facilities, provided you maintain
    clear directions next to the object code saying where to find the
    Corresponding Source.  Regardless of what server hosts the
    Corresponding Source, you remain obligated to ensure that it is
    available for as long as needed to satisfy these requirements.

    e) Convey the object code using peer-to-peer transmission, provided
    you inform other peers where the object code and Corresponding
    Source of the work are being offered to the general public at no
    charge under subsection 6d.

  A separable portion of the object code, whose source code is excluded
from the Corresponding Source as a System Library, need not be
included in conveying the object code work.

  A "User Product" is either (1) a "consumer product", which means any
tangible personal property which is normally used for personal, family,
or household purposes, or (2) anything designed or sold for incorporation
into a dwelling.  In determining whether a product is a consumer product,
doubtful cases shall be resolved in favor of coverage.  For a particular
product received by a particular user, "normally used" refers to a
typical or common use of that class of product, regardless of the status
of the particular user or of the way in which the particular user
actually uses, or expects or is expected to use, the product.  A product
is a consumer product regardless of whether the product has substantial
commercial, industrial or non-consumer uses, unless such uses represent
the only significant mode of use of the product.

  "Installation Information" for a User Product means any methods,
procedures, authorization keys, or other information required to install
and execute modified versions of a covered work in that User Product from
a modified version of its Corresponding Source.  The information must
suffice to ensure that the continued functioning of the modified object
code is in no case prevented or interfered with solely because
modification has been made.

  If you convey an object code work under this section in, or with, or
specifically for use in, a User Product, and the conveying occurs as
part of a transaction in which the right of possession and use of the
User Product is transferred to the recipient in perpetuity or for a
fixed term (regardless of how the transaction is characterized), the
Corresponding Source conveyed under this section must be accompanied
by the Installation Information.  But this requirement does not apply
if neither you nor any third party retains the ability to install
modified object code on the User Product (for example, the work has
been installed in ROM).

  The requirement to provide Installation Information does not include a
requirement to continue to provide support service, warranty, or updates
for a work that has been modified or installed by the recipient, or for
the User Product in which it has been modified or installed.  Access to a
network may be denied when the modification itself materially and
adversely affects the operation of the network or violates the rules and
protocols for communication across the network.

  Corresponding Source conveyed, and Installation Information provided,
in accord with this section must be in a format that is publicly
documented (and with an implementation available to the public in
source code form), and must require no special password or key for
unpacking, reading or copying.

  7. Additional Terms.

  "Additional permissions" are terms that supplement the terms of this
License by making exceptions from one or more of its conditions.
Additional permissions that are applicable to the entire Program shall
be treated as though they were included in this License, to the extent
that they are valid under applicable law.  If additional permissions
apply only to part of the Program, that part may be used separately
under those permissions, but the entire Program remains governed by
this License without regard to the additional permissions.

  When you convey a copy of a covered work, you may at your option
remove any additional permissions from that copy, or from any part of
it.  (Additional permissions may be written to require their own
removal in certain cases when you modify the work.)  You may place
additional permissions on material, added by you to a covered work,
for which you have or can give appropriate copyright permission.

  Notwithstanding any other provision of this License, for material you
add to a covered work, you may (if authorized by the copyright holders of
that material) supplement the terms of this License with terms:

    a) Disclaiming warranty or limiting liability differently from the
    terms of sections 15 and 16 of this License; or

    b) Requiring preservation of specified reasonable legal notices or
    author attributions in that material or in the Appropriate Legal
    Notices displayed by works containing it; or

    c) Prohibiting misrepresentation of the origin of that material, or
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version; or

    d) Limiting the use for publicity purposes of names of licensors or
    authors of the material; or

    e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks; or

    f) Requiring indemnification of licensors and authors of that
    material by anyone who conveys the material (or modified versions of
    it) with contractual assumptions of liability to the recipient, for
    any liability that these contractual assumptions directly impose on
    those licensors and authors.

  All other non-permissive additional terms are considered "further
restrictions" within the meaning of section 10.  If the Program as you
received it, or any part of it, contains a notice stating that it is
governed by this License along with a term that is a further
restriction, you may remove that term.  If a license document contains
a further restriction but permits relicensing or conveying under this
License, you may add to a covered work material governed by the terms
of that license document, provided that the further restriction does
not survive such relicensing or conveying.

  If you add terms to a covered work in accord with this section, you
must place, in the relevant source files, a statement of the
additional terms that apply to those files, or a notice indicating
where to find the applicable terms.

  Additional terms, permissive or non-permissive, may be stated in the
form of a separately written license, or stated as exceptions;
the above requirements apply either way.

  8. Termination.

  You may not propagate or modify a covered work except as expressly
provided under this License.  Any attempt otherwise to propagate or
modify it is void, and will automatically terminate your rights under
this License (including any patent licenses granted under the third
paragraph of section 11).

  However, if you cease all violation of this License, then your
license from a particular copyright holder is reinstated (a)
provisionally, unless and until the copyright holder explicitly and
finally terminates your license, and (b) permanently, if the copyright
holder fails to notify you of the violation by some reasonable means
prior to 60 days after the cessation.

  Moreover, your license from a particular copyright holder is
reinstated permanently if the copyright holder notifies you of the
violation by some reasonable means, this is the first time you have
received notice of violation of this License (for any work) from that
copyright holder, and you cure the violation prior to 30 days after
your receipt of the notice.

  Termination of your rights under this section does not terminate the
licenses of parties who have received copies or rights from you under
this License.  If your rights have been terminated and not permanently
reinstated, you do not qualify to receive new licenses for the same
material under section 10.

  9. Acceptance Not Required for Having Copies.

  You are not required to accept this License in order to receive or
run a copy of the Program.  Ancillary propagation of a covered work
occurring solely as a consequence of using peer-to-peer transmission
to receive a copy likewise does not require acceptance.  However,
nothing other than this License grants you permission to propagate or
modify any covered work.  These actions infringe copyright if you do
not accept this License.  Therefore, by modifying or propagating a
covered work, you indicate your acceptance of this License to do so.

  10. Automatic Licensing of Downstream Recipients.

  Each time you convey a covered work, the recipient automatically
receives a license from the original licensors, to run, modify and
propagate that work, subject to this License.  You are not responsible
for enforcing compliance by third parties with this License.

  An "entity transaction" is a transaction transferring control of an
organization, or substantially all assets of one, or subdividing an
organization, or merging organizations.  If propagation of a covered
work results from an entity transaction, each party to that
transaction who receives a copy of the work also receives whatever
licenses to the work the party's predecessor in interest had or could
give under the previous paragraph, plus a right to possession of the
Corresponding Source of the work from the predecessor in interest, if
the predecessor has it or can get it with reasonable efforts.

  You may not impose any further restrictions on the exercise of the
rights granted or affirmed under this License.  For example, you may
not impose a license fee, royalty, or other charge for exercise of
rights granted under this License, and you may not initiate litigation
(including a cross-claim or counterclaim in a lawsuit) alleging that
any patent claim is infringed by making, using, selling, offering for
sale, or importing the Program or any portion of it.

  11. Patents.

  A "contributor" is a copyright holder who authorizes use under this
License of the Program or a work on which the Program is based.  The
work thus licensed is called the contributor's "contributor version".

  A contributor's "essential patent claims" are all patent claims
owned or controlled by the contributor, whether already acquired or
hereafter acquired, that would be infringed by some manner, permitted
by this License, of making, using, or selling its contributor version,
but do not include claims that would be infringed only as a
consequence of further modification of the contributor version.  For
purposes of this definition, "control" includes the right to grant
patent sublicenses in a manner consistent with the requirements of
this License.

  Each contributor grants you a non-exclusive, worldwide, royalty-free
patent license under the contributor's essential patent claims, to
make, use, sell, offer for sale, import and otherwise run, modify and
propagate the contents of its contributor version.

  In the following three paragraphs, a "patent license" is any express
agreement or commitment, however denominated, not to enforce a patent
(such as an express permission to practice a patent or covenant not to
sue for patent infringement).  To "grant" such a patent license to a
party means to make such an agreement or commitment not to enforce a
patent against the party.

  If you convey a covered work, knowingly relying on a patent license,
and the Corresponding Source of the work is not available for anyone
to copy, free of charge and under the terms of this License, through a
publicly available network server or other readily accessible means,
then you must either (1) cause the Corresponding Source to be so
available, or (2) arrange to deprive yourself of the benefit of the
patent license for this particular work, or (3) arrange, in a manner
consistent with the requirements of this License, to extend the patent
license to downstream recipients.  "Knowingly relying" means you have
actual knowledge that, but for the patent license, your conveying the
covered work in a country, or your recipient's use of the covered work
in a country, would infringe one or more identifiable patents in that
country that you have reason to believe are valid.

  If, pursuant to or in connection with a single transaction or
arrangement, you convey, or propagate by procuring conveyance of, a
covered work, and grant a patent license to some of the parties
receiving the covered work authorizing them to use, propagate, modify
or convey a specific copy of the covered work, then the patent license
you grant is automatically extended to all recipients of the covered
work and works based on it.

  A patent license is "discriminatory" if it does not include within
the scope of its coverage, prohibits the exercise of, or is
conditioned on the non-exercise of one or more of the rights that are
specifically granted under this License.  You may not convey a covered
work if you are a party to an arrangement with a third party that is
in the business of distributing software, under which you make payment
to the third party based on the extent of your activity of conveying
the work, and under which the third party grants, to any of the
parties who would receive the covered work from you, a discriminatory
patent license (a) in connection with copies of the covered work
conveyed by you (or copies made from those copies), or (b) primarily
for and in connection with specific products or compilations that
contain the covered work, unless you entered into that arrangement,
or that patent license was granted, prior to 28 March 2007.

  Nothing in this License shall be construed as excluding or limiting
any implied license or other defenses to infringement that may
otherwise be available to you under applicable patent law.

  12. No Surrender of Others' Freedom.

  If conditions are imposed on you (whether by court order, agreement or
otherwise) that contradict the conditions of this License, they do not
excuse you from the conditions of this License.  If you cannot convey a
covered work so as to satisfy simultaneously your obligations under this
License and any other pertinent obligations, then as a consequence you may
not convey it at all.  For example, if you agree to terms that obligate you
to collect a royalty for further conveying from those to whom you convey
the Program, the only way you could satisfy both those terms and this
License would be to refrain entirely from conveying the Program.

  13. Use with the GNU Affero General Public License.

  Notwithstanding any other provision of this License, you have
permission to link or combine any covered work with a work licensed
under version 3 of the GNU Affero General Public License into a single
combined work, and to convey the resulting work.  The terms of this
License will continue to apply to the part which is the covered work,
but the special requirements of the GNU Affero General Public License,
section 13, concerning interaction through a network will apply to the
combination as such.

  14. Revised Versions of this License.

  The Free Software Foundation may publish revised and/or new versions of
the GNU General Public License from time to time.  Such new versions will
be similar in spirit to the present version, but may differ in detail to
address new problems or concerns.

  Each version is given a distinguishing version number.  If the
Program specifies that a certain numbered version of the GNU General
Public License "or any later version" applies to it, you have the
option of following the terms and conditions either of that numbered
version or of any later version published by the Free Software
Foundation.  If the Program does not specify a version number of the
GNU General Public License, you may choose any version ever published
by the Free Software Foundation.

  If the Program specifies that a proxy can decide which future
versions of the GNU General Public License can be used, that proxy's
public statement of acceptance of a version permanently authorizes you
to choose that version for the Program.

  Later license versions may give you additional or different
permissions.  However, no additional obligations are imposed on any
author or copyright holder as a result of your choosing to follow a
later version.

  15. Disclaimer of Warranty.

  THERE IS NO WARRANTY FOR THE PROGRAM, TO THE EXTENT PERMITTED BY
APPLICABLE LAW.  EXCEPT WHEN OTHERWISE STATED IN WRITING THE COPYRIGHT
HOLDERS AND/OR OTHER PARTIES PROVIDE THE PROGRAM "AS IS" WITHOUT WARRANTY
OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING, BUT NOT LIMITED TO,
THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
PURPOSE.  THE ENTIRE RISK AS TO THE QUALITY AND PERFORMANCE OF THE PROGRAM
IS WITH YOU.  SHOULD THE PROGRAM PROVE DEFECTIVE, YOU ASSUME THE COST OF
ALL NECESSARY SERVICING, REPAIR OR CORRECTION.

  16. Limitation of Liability.

  IN NO EVENT UNLESS REQUIRED BY APPLICABLE LAW OR AGREED TO IN WRITING
WILL ANY COPYRIGHT HOLDER, OR ANY OTHER PARTY WHO MODIFIES AND/OR CONVEYS
THE PROGRAM AS PERMITTED ABOVE, BE LIABLE TO YOU FOR DAMAGES, INCLUDING ANY
GENERAL, SPECIAL, INCIDENTAL OR CONSEQUENTIAL DAMAGES ARISING OUT OF THE
USE OR INABILITY TO USE THE PROGRAM (INCLUDING BUT NOT LIMITED TO LOSS OF
DATA OR DATA BEING RENDERED INACCURATE OR LOSSES SUSTAINED BY YOU OR THIRD
PARTIES OR A FAILURE OF THE PROGRAM TO OPERATE WITH ANY OTHER PROGRAMS),
EVEN IF SUCH HOLDER OR OTHER PARTY HAS BEEN ADVISED OF THE POSSIBILITY OF
SUCH DAMAGES.

  17. Interpretation of Sections 15 and 16.

  If the disclaimer of warranty and limitation of liability provided
above cannot be given local legal effect according to their terms,
reviewing courts shall apply local law that most closely approximates
an absolute waiver of all civil liability in connection with the
Program, unless a warranty or assumption of liability accompanies a
copy of the Program in return for a fee.

                     END OF TERMS AND CONDITIONS

            How to Apply These Terms to Your New Programs

  If you develop a new program, and you want it to be of the greatest
possible use to the public, the best way to achieve this is to make it
free software which everyone can redistribute and change under these terms.

  To do so, attach the following notices to the program.  It is safest
to attach them to the start of each source file to most effectively
state the exclusion of warranty; and each file should have at least
the "copyright" line and a pointer to where the full notice is found.

    <one line to give the program's name and a brief idea of what it does.>
    Copyright (C) <year>  <name of author>

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.

Also add information on how to contact you by electronic and paper mail.

  If the program does terminal interaction, make it output a short
notice like this when it starts in an interactive mode:

    <program>  Copyright (C) <year>  <name of author>
    This program comes with ABSOLUTELY NO WARRANTY; for details type `show w'.
    This is free software, and you are welcome to redistribute it
    under certain conditions; type `show c' for details.

The hypothetical commands `show w' and `show c' should show the appropriate
parts of the General Public License.  Of course, your program's commands
might be different; for a GUI interface, you would use an "about box".

  You should also get your employer (if you work as a programmer) or school,
if any, to sign a "copyright disclaimer" for the program, if necessary.
For more information on this, and how to apply and follow the GNU GPL, see
<https://www.gnu.org/licenses/>.

  The GNU General Public License does not permit incorporating your program
into proprietary programs.  If your program is a subroutine library, you
may consider it more useful to permit linking proprietary applications with
the library.  If this is what you want to do, use the GNU Lesser General
Public License instead of this License.  But first, please read
<https://www.gnu.org/licenses/why-not-lgpl.html>.
